In [ ]:
import cartopy.crs as ccrs
import matplotlib.animation as animation
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt

# convert the animation to javascript for display
#plt.rcParams["animation.html"] = "jshtml"

import geocat.viz.util as gvutil

from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

plt.rcParams['animation.embed_limit'] = 2**128

import datetime

from datetime import date
import pandas as pd


from metpy.plots import colortables

In [ ]:
min_lon = -140.
max_lon =   10.
delta_lon = 20.
min_lat = 0.
max_lat =  30.
central_lon = .5*(min_lon + max_lon)

# set the min and max values to shade in the IR brightness temperature field
Cmin=  170
Cmax = 300
Cint = 10


In [ ]:

def animate(i,date,min_lon,max_lon,min_lat,max_lat,Cmin,Cmax):
    central_lon = .5*(min_lon + max_lon)
    delta_lon = 20.
    datePlot = date + pd.Timedelta(days=i*.25)
    Y = datePlot.strftime('%Y')
    
    url_base = 'https://www.ncei.noaa.gov/thredds/dodsC/cdr/gridsat/' + Y + '/'  
    #url_base = '/home/sagan/work/data/data100/data/gridsat/data/'

    url = url_base + "GRIDSAT-B1."  + datePlot.strftime('%Y.%m.%d.%H')  + ".v02r01.nc"
    cmap = colortables.get_colortable('ir_tv1')
    print(" here ", i , url )
    ds = xr.open_dataset(url)
    LatIndexer, LonIndexer = 'lat', 'lon'
    irwin = ds.irwin_cdr[0,:,:].sel(**{LatIndexer: slice(min_lat, max_lat),
                        LonIndexer: slice(min_lon, max_lon)})
    print(irwin.min().values)
    ct=plt.imshow(irwin[:-1,:-1], cmap=cmap, origin='lower', vmin=Cmin,vmax=Cmax,extent=[min_lon, max_lon,min_lat, max_lat])
 
    
    
    #ct = plt.pcolormesh(lons, lats, irwin[:-1,:-1],cmap='Greys', rasterized=True, vmin=Cmin, vmax =Cmax, shading='flat')
    plt.title(datePlot, y=1.2)
    

In [ ]:
year =  2018
month = 9
day = 1
hour = 0
ntimes = 4*30

# encode the date 
date = datetime.datetime(year,month,day,hour)


In [ ]:
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()

transform = ccrs.PlateCarree()

#fig = plt.figure(figsize=(18,6))
fig = plt.figure(figsize=(8,4))

# Generate axes, using Cartopy, drawing coastlines, and adding features
projection = ccrs.PlateCarree()
ax = plt.axes(projection=projection)
ax.coastlines(linewidths=0.75, color='black')

i=0
Y = date.strftime('%Y')

url_base = 'https://www.ncei.noaa.gov/thredds/dodsC/cdr/gridsat/' + Y + '/'
#url_base = '/home/sagan/work/data/data100/data/gridsat/data/'



url = url_base + "GRIDSAT-B1."  + date.strftime('%Y.%m.%d.%H')  + ".v02r01.nc"
cmap = colortables.get_colortable('ir_tv1')
print(" here ", i , url )
ds = xr.open_dataset(url)

LatIndexer, LonIndexer = 'lat', 'lon'
irwin = ds.irwin_cdr[0,:,:].sel(**{LatIndexer: slice(min_lat, max_lat),
                    LonIndexer: slice(min_lon, max_lon)})
ct=plt.imshow(irwin[:-1,:-1], cmap=cmap, origin='lower', vmin=Cmin,vmax=Cmax,extent=[min_lon, max_lon,min_lat, max_lat])

print(irwin.min().values)
# Add color bar
cbar_ticks = np.arange(Cmin,Cmax,Cint)
cbar = plt.colorbar(ct,
                    orientation='horizontal',
                    shrink=0.95 ,aspect=30,
                    pad=0.15,
                    extendrect=True,
                    ticks=cbar_ticks)

cbar.ax.tick_params(labelsize=10)  

# Use geocat.viz.util convenience function to set axes limit
gvutil.set_axes_limits_and_ticks(ax,
                                xlim=(min_lon, max_lon),
                                ylim=(min_lat, max_lat),
                                xticks=np.linspace(min_lon,max_lon, 5),
                                yticks=np.linspace(min_lat,max_lat, 5))

# Use geocat.viz.util convenience function to add major tick lines
gvutil.add_major_minor_ticks(ax, y_minor_per_major=1, labelsize=12)

# Use geocat.viz.util convenience function to add lat and lon tick labels
gvutil.add_lat_lon_ticklabels(ax)

# Remove degree symbol from tick label
ax.yaxis.set_major_formatter(LatitudeFormatter(degree_symbol=''))
ax.xaxis.set_major_formatter(LongitudeFormatter(degree_symbol=''))
# Use geocat.viz.util convenience function to set titles and labels

gvutil.set_titles_and_labels(ax,
                                 righttitle="K",
                                 righttitlefontsize=15,
                                 lefttitle="11 micron BT",
                                 lefttitlefontsize=15,
                                 xlabel="",
                                 ylabel="")



#---------------------------------------------------------------------------------------------------
animArgs = (date,min_lon,max_lon,min_lat,max_lat,Cmin,Cmax)
# runs the animation initiated with the frame from init and progressed with the animate function
anim = animation.FuncAnimation(fig,animate,frames=ntimes, interval=200, fargs=animArgs)





anim    